# Loading packages and intial dataset

## Standard packages

In [104]:
import numpy as np
import pandas as pd
import importlib
import matplotlib.pyplot as plt

import warnings
warnings.simplefilter('ignore')

## Bespoke packages

In [173]:
import word_parser
importlib.reload(word_parser)
from word_parser import infer_spaces, separate_merged_words, pairwise
from cleaner import fix_shortened_makes
from helper import one_hot_encode, add_one_hot_encoding
from collections import Counter

In [3]:
df_raw = pd.read_csv('data/error_codes_only.csv')

/Users/nicholashirons/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (21,24,30,31,35,36,65,66,116,117,122,127,131,136,137,167,169,170,173,174,179) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [9]:
df = df_raw[['Make']]

## Part 1: Cleaning text data to create categorical target variables

In [58]:
df['op_descriptions'] = [separate_merged_words(row) if type(row) == str else ''
                         for row in df_raw['Operation Code Descriptions']]

/Users/nicholashirons/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [59]:
most_common_pairs = pd.DataFrame(Counter([pair for sublist in df['op_descriptions']
                                          if type(sublist) == str
                                          for pair in pairwise(sublist.split())]).most_common())

In [94]:
most_common_pairs.head(10)

,0,1
0,"(this, visit)",8089
1,"(on, this)",8051
2,"(ok, on)",8003
3,"(multi, point)",6021
4,"(point, inspection)",6021
5,"(inspected, and)",5786
6,"(and, ok)",4680
7,"(perform, multi)",3891
8,"(battery, test)",3870
9,"(tires, inspected)",3640


In [148]:
operations = ['multi point inspection',
              'battery test',
              'tire inflation',
              'hazardous waste',
              'change oil',
              'filter',
              'belts and hoses',
              'air filter',
              'safety systems',
              'nitro fill']

In [149]:
for op in operations:
    df[op] = df['op_descriptions'].str.contains(op)
    df[op] = df[op].astype(int)

How many descriptions do these 9 strings capture?

In [150]:
pd.DataFrame(df[operations].sum(axis = 1).value_counts(), columns = ['count'])

,count
0,6089
1,3653
2,3233
3,1326
4,1262
5,341
6,289


We still have over 6000 undefined strings.

In [151]:
undefined_descriptions = [row for row in df['op_descriptions'] if not any(op in row for op in operations)]

In [152]:
most_common_other_pairs = pd.DataFrame(Counter([pair for sublist in undefined_descriptions
                                          if type(sublist) == str
                                          for pair in pairwise(sublist.split())]).most_common())

In [153]:
most_common_other_pairs.head(10)

,0,1
0,"(per, estimate)",1365
1,"(repair, per)",715
2,"(refinish, per)",545
3,"(estimate, refinish)",479
4,"(prep, vehicle)",463
5,"(no, charge)",451
6,"(clean, vehicle)",425
7,"(csi, inspection)",413
8,"(inspection, fee)",407
9,"(no, work)",390


In [154]:
len([row for row in df['op_descriptions'] if ('repair' in row or 'refinish' in row)])

2047

In [155]:
df['repair or refinish'] = ((df['op_descriptions'].str.contains('repair') > 0) |
                            (df['op_descriptions'].str.contains('refinish') > 0)).astype(int)

In [156]:
operations.append('repair or refinish')

In [157]:
pd.DataFrame(df[operations].sum(axis = 1).value_counts(), columns = ['count'])

,count
0,5138
1,4162
2,3320
3,1536
4,1289
5,447
6,277
7,24


In [158]:
df.head()

,Make,op_descriptions,bespoke,multi point inspection,battery test,tire inflation,hazardous waste,change oil,filter,belts and hoses,air filter,safety systems,nitro fill,repair or refinish
0,MITSUBISHI FUSO,replaced fuel filter t test drove perform mult...,0,1,1,0,0,0,1,0,0,0,0,0
1,FORD,no work done no work done no work done brake i...,1,0,0,0,0,0,1,0,0,0,0,0
2,FORD,perform the works package per coupon change oi...,0,1,1,0,0,1,1,1,1,0,0,0
3,FORD,mount and balance four tires with four tire pu...,0,1,1,0,0,0,1,0,1,0,0,1
4,FORD,spray in bed liner repair per estimate,0,0,0,0,0,0,0,0,0,0,0,1


In [159]:
df['bespoke'] = (df[operations].sum(axis = 1) == 0).astype(int)

Add makes

In [162]:
df['Make'] = fix_shortened_makes(df['Make'])

In [175]:
df = add_one_hot_encoding(df, ['Make'], rank_threshold = np.inf, count_threshold = 100)

Add engine size

In [180]:
df['engine_size'] = df_raw['VIN Explosion Engine Size']

In [242]:
df['engine_size'] = [float(item[0]) if ((type(item) == list) and (len(item) > 0))
 else np.nan for item in df['engine_size'].str.findall("\d+\.\d+")]

Add fuel type

In [183]:
df['fuel'] = df_raw['VIN Explosion Fuel Type']

In [193]:
df = add_one_hot_encoding(df, ['fuel'], rank_threshold = 5)

In [194]:
df['fuel_ELEC OR HYBRID'] = df['fuel_ELECTRIC'] | df['fuel_HYBRID']

Add mileage

In [188]:
df['mileage'] = df_raw['Mileage Out']

Add date and convert to season in which error code was received

In [196]:
df['date'] = df_raw['Open Date']

In [198]:
df['date'] = pd.to_datetime(df['date'], infer_datetime_format = True)

In [200]:
df['appointment_month'] = df['date'].dt.month

In [201]:
df['season'] = df['appointment_month'].map({1.0: 'winter',
                                                  2.0: 'winter',
                                                  3.0: 'spring',
                                                  4.0: 'spring',
                                                  5.0: 'spring',
                                                  6.0: 'summer',
                                                  7.0: 'summer',
                                                  8.0: 'summer',
                                                  9.0: 'fall',
                                                  10.0: 'fall',
                                                  11.0: 'fall',
                                                  12.0: 'winter'
                                                  })

In [203]:
df = add_one_hot_encoding(df, ['season'], rank_threshold = 4)

Add car age

In [204]:
df['year'] = df_raw['Year']

In [233]:
df['year'] = np.where(df['year'] < 1900, df['year'] + 2000, df['year'])
df['year'] = np.where(df['year'] > 2020, df['year'] - 100, df['year'])

In [234]:
df['age'] = df['date'].dt.year - df['year'] + 1

Clean and rename columns

In [237]:
df.columns = [col.lower().replace(' ', '_') for col in df.columns]

In [244]:
df.to_csv('data/all.csv')

In [247]:
target_cols = [col.replace(' ', '_') for col in operations]

In [253]:
df = df.rename(columns = {col:'y_'+col for col in target_cols})